In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
from matplotlib import pyplot as plt 
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import keras
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [ ]:
#원 데이터 가져오기
data_jj=pd.read_csv('/content/drive/My Drive/data/dacon_data/201901-202003.csv')

In [ ]:
#sub 데이터 가져오기
sub=pd.read_csv('/content/drive/My Drive/data/dacon_data/submission.csv')

In [ ]:
data_jj.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [ ]:
dd=data_jj[['AMT']]

In [ ]:
data_jj['REG_YYMM']

0           201901
1           201901
2           201901
3           201901
4           201901
             ...  
24697787    202003
24697788    202003
24697789    202003
24697790    202003
24697791    202003
Name: REG_YYMM, Length: 24697792, dtype: int64

# 외부 데이터 +

## 날씨
출처 : https://data.kma.go.kr/climate/StatisticsDivision/selectStatisticsDivision.do?pgmNo=158

In [ ]:
#REG_YYMM
list_2010=[' 2010-01',' 2010-02',' 2010-03',' 2010-04',' 2010-05',' 2010-06',' 2010-07',' 2010-08',' 2010-09',' 2010-10',' 2010-11',' 2010-12']
list_2011=[' 2011-01',' 2011-02',' 2011-03',' 2011-04',' 2011-05',' 2011-06',' 2011-07',' 2011-08',' 2011-09',' 2011-10',' 2011-11',' 2011-12']
list_2012=[' 2012-01',' 2012-02',' 2012-03',' 2012-04',' 2012-05',' 2012-06',' 2012-07',' 2012-08',' 2012-09',' 2012-10',' 2012-11',' 2012-12']
list_2013=[' 2013-01',' 2013-02',' 2013-03',' 2013-04',' 2013-05',' 2013-06',' 2013-07',' 2013-08',' 2013-09',' 2013-10',' 2013-11',' 2013-12']
list_2014=[' 2014-01',' 2014-02',' 2014-03',' 2014-04',' 2014-05',' 2014-06',' 2014-07',' 2014-08',' 2014-09',' 2014-10',' 2014-11',' 2014-12']
list_2015=[' 2015-01',' 2015-02',' 2015-03',' 2015-04',' 2015-05',' 2015-06',' 2015-07',' 2015-08',' 2015-09',' 2015-10',' 2015-11',' 2015-12']
list_2016=[' 2016-01',' 2016-02',' 2016-03',' 2016-04',' 2016-05',' 2016-06',' 2016-07',' 2016-08',' 2016-09',' 2016-10',' 2016-11',' 2016-12']
list_2017=[' 2017-01',' 2017-02',' 2017-03',' 2017-04',' 2017-05',' 2017-06',' 2017-07',' 2017-08',' 2017-09',' 2017-10',' 2017-11',' 2017-12']
list_2018=[' 2018-01',' 2018-02',' 2018-03',' 2018-04',' 2018-05',' 2018-06',' 2018-07',' 2018-08',' 2018-09',' 2018-10',' 2018-11',' 2018-12']
list_2019=[' 2019-01',' 2019-02',' 2019-03',' 2019-04',' 2019-05',' 2019-06',' 2019-07',' 2019-08',' 2019-09',' 2019-10',' 2019-11',' 2019-12']
list_2020=[' 2020-01',' 2020-02',' 2020-03',' 2020-04',' 2020-05',' 2020-06',' 2020-07',' 2020-08',' 2020-09',' 2020-10',' 2020-11',' 2020-12']

In [ ]:
#list 맞춰주기 - 문제있을 시 얘기 좀.
rain_list=['강원','경기','경남','경북','광주','대구','대전','부산','서울','세종','울산','인천','전남','전북','제주','충남','충북']
#강수량 데이터프레임 만들기 코드
dp_zero_rain=pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','RAIN_DAY'])
g=''
for r in rain_list:
  if r == '경기':
    g='서울'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r == '광주':
    g='전남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r == '대구':
    g='경남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r == '대전':
    g='충남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r=='부산':
    g='경남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r=='세종':
    g='충남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r=='울산':
    g='경남'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  elif r=='인천':
    g='서울'
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(g),encoding="cp949")
  else:
    d_rain=pd.read_csv('/content/drive/My Drive/data/weather2/rain/{}_강수일수.csv'.format(r),encoding="cp949")
  d_2010=[]
  d_2011=[]
  d_2012=[]
  d_2013=[]
  d_2014=[]
  d_2015=[]
  d_2016=[]
  d_2017=[]
  d_2018=[]
  d_2019=[]
  d_2020=[]
  d_sido=[]
  for i in range(1,13):
    d_2010.append(d_rain[d_rain.iloc[:,0]=='2010'].iloc[0,i])
    d_2011.append(d_rain[d_rain.iloc[:,0]=='2011'].iloc[0,i])
    d_2012.append(d_rain[d_rain.iloc[:,0]=='2012'].iloc[0,i])
    d_2013.append(d_rain[d_rain.iloc[:,0]=='2013'].iloc[0,i])
    d_2014.append(d_rain[d_rain.iloc[:,0]=='2014'].iloc[0,i])
    d_2015.append(d_rain[d_rain.iloc[:,0]=='2015'].iloc[0,i])
    d_2016.append(d_rain[d_rain.iloc[:,0]=='2016'].iloc[0,i])
    d_2017.append(d_rain[d_rain.iloc[:,0]=='2017'].iloc[0,i])
    d_2018.append(d_rain[d_rain.iloc[:,0]=='2018'].iloc[0,i])
    d_2019.append(d_rain[d_rain.iloc[:,0]=='2019'].iloc[0,i])
    d_2020.append(d_rain[d_rain.iloc[:,0]=='2020'].iloc[0,i])
    d_sido.append(r)
  dp_2010=pd.DataFrame({'REG_YYMM':list_2010,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2010})
  dp_2011=pd.DataFrame({'REG_YYMM':list_2011,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2011})
  dp_2012=pd.DataFrame({'REG_YYMM':list_2012,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2012})
  dp_2013=pd.DataFrame({'REG_YYMM':list_2013,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2013})
  dp_2014=pd.DataFrame({'REG_YYMM':list_2014,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2014})
  dp_2015=pd.DataFrame({'REG_YYMM':list_2015,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2015})
  dp_2016=pd.DataFrame({'REG_YYMM':list_2016,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2016})
  dp_2017=pd.DataFrame({'REG_YYMM':list_2017,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2017})
  dp_2018=pd.DataFrame({'REG_YYMM':list_2018,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2018})
  dp_2019=pd.DataFrame({'REG_YYMM':list_2019,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2019})
  dp_2020=pd.DataFrame({'REG_YYMM':list_2020,'CARD_SIDO_NM':d_sido,'RAIN_DAY':d_2020})
  dp_full=pd.concat([dp_2010,dp_2011,dp_2012,dp_2013,dp_2014,dp_2015,dp_2016,dp_2017,dp_2018,dp_2019,dp_2020[:7]],axis=0,ignore_index=True)
  # dp_full.to_csv('/content/drive/My Drive/data/weather/rain/{}_RAINDAY.csv'.format(r),index=False, encoding='utf-8-sig')
  dp_zero_rain=pd.concat([dp_zero_rain,dp_full],axis=0,ignore_index=True)
#dp_zero_rain.to_csv('/content/drive/My Drive/data/weather2/rain/K_RAINDAY.csv'.format(),index=False, encoding='utf-8-sig')  

In [ ]:
weather_list=['강원','경기','경남','경북','광주','대구','대전','부산','서울','세종','울산','인천','전남','전북','제주','충남','충북']

In [ ]:
#습도 데이터프레임 만들기 코드
dp_zero_humid=pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','HUMID_AVG','HUMID_MIN'])
for w in weather_list:
  d_weather=pd.read_csv('/content/drive/My Drive/data/weather2/humid/{}_습도.csv'.format(w),encoding="cp949")
  d_nan=pd.read_csv('/content/drive/My Drive/data/weather2/humid/{}_습도.csv'.format('충남'),encoding="cp949")
  if w =='세종':
    d_weather_wanted=pd.concat([d_nan[:d_nan[d_nan['일시']==' 2019-06'].index[0]],d_weather[1:-1]])
  else:
    d_weather_wanted=d_weather.iloc[:-1,:] 
  d_weather_wanted.columns=['REG_YYMM','HUMID_AVG','HUMID_MIN']
  d_weather_wanted['CARD_SIDO_NM']=w
  d_weather_wanted=d_weather_wanted[['REG_YYMM','CARD_SIDO_NM','HUMID_AVG','HUMID_MIN']]
  # d_weather_wanted.to_csv('/content/drive/My Drive/data/weather/humid/{}_HUMID.csv'.format(w),index=False, encoding='utf-8-sig')
  dp_zero_humid=pd.concat([dp_zero_humid,d_weather_wanted],axis=0,ignore_index=True)
dp_zero_humid.to_csv('/content/drive/My Drive/data/weather2/humid/K_HUMID.csv'.format(),index=False, encoding='utf-8-sig')  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
#일사량 데이터프레임 만들기 코드
dp_zero_sun=pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','SUN_HOUR_SUM'])
for w in weather_list:
  d_weather=pd.read_csv('/content/drive/My Drive/data/weather2/sun/{}_일조.csv'.format(w),encoding="cp949")
  d_nan=pd.read_csv('/content/drive/My Drive/data/weather2/sun/{}_일조.csv'.format('충남'),encoding="cp949")
  if w =='세종':
    d_weather=pd.concat([d_nan[:d_nan[d_nan['일시']==' 2019-06'].index[0]],d_weather[1:-1]])
    d_weather_wanted=d_weather.iloc[:,:2]
  else:
    d_weather_wanted=d_weather.iloc[:-1,:2] 
  d_weather_wanted.columns=['REG_YYMM','SUN_HOUR_SUM']
  d_weather_wanted['CARD_SIDO_NM']=w
  d_weather_wanted=d_weather_wanted[['REG_YYMM','CARD_SIDO_NM','SUN_HOUR_SUM']]
  # d_weather_wanted.to_csv('/content/drive/My Drive/data/weather/sun/{}_SUN.csv'.format(w),index=False, encoding='utf-8-sig')
  dp_zero_sun=pd.concat([dp_zero_sun,d_weather_wanted],axis=0,ignore_index=True)
dp_zero_sun.to_csv('/content/drive/My Drive/data/weather2/sun/K_SUN.csv'.format(),index=False, encoding='utf-8-sig')  

In [ ]:
#기온 데이터프레임 만들기 코드
dp_zero_temp=pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','TEMP_AVG','TEMP_TOP','TEMP_BOT','RAIN_AVG'])
for w in weather_list:
  d_weather=pd.read_csv('/content/drive/My Drive/data/weather2/temp/{}_기온.csv'.format(w),encoding="cp949")
  d_nan=pd.read_csv('/content/drive/My Drive/data/weather2/temp/{}_기온.csv'.format('충남'),encoding="cp949")
  if w =='세종':
    d_weather=pd.concat([d_nan[:d_nan[d_nan['일시']==' 2019-06'].index[0]],d_weather[1:-1]])
    d_weather_wanted=d_weather.iloc[:,:5]
  else:
    d_weather_wanted=d_weather.iloc[:-1,:5] 
  d_weather_wanted.columns=['REG_YYMM','TEMP_AVG','TEMP_TOP','TEMP_BOT','RAIN_AVG']
  d_weather_wanted['CARD_SIDO_NM']=w
  d_weather_wanted=d_weather_wanted[['REG_YYMM','CARD_SIDO_NM','TEMP_AVG','TEMP_TOP','TEMP_BOT','RAIN_AVG']]
  # d_weather_wanted.to_csv('/content/drive/My Drive/data/weather2/temp/{}_TEMP.csv'.format(w),index=False, encoding='utf-8-sig')
  dp_zero_temp=pd.concat([dp_zero_temp,d_weather_wanted],axis=0,ignore_index=True)
dp_zero_temp.to_csv('/content/drive/My Drive/data/weather2/temp/K_TEMP.csv'.format(),index=False, encoding='utf-8-sig')  

In [ ]:
dp_all=dp_zero_temp.merge(dp_zero_sun).merge(dp_zero_humid).merge(dp_zero_rain)

In [ ]:
#불쾌지수 구하기
dp_all['THI_SCORE']=((9/5)*dp_all['TEMP_AVG'])-(0.55*(1-dp_all['HUMID_AVG']/100)*(((9/5)*dp_all['TEMP_AVG'])-26))+32

In [ ]:
dp_all

,REG_YYMM,CARD_SIDO_NM,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE
0,2010-01,강원,-4.3,1.2,-9.8,58.8,199.2,72,22,7.0,29.456
1,2010-02,강원,0.1,5.1,-4.5,113.5,145.5,70,15,10.7,36.4403
2,2010-03,강원,3.1,7.3,-0.8,172.5,135.2,76,17,15.2,40.2754
3,2010-04,강원,8.6,14.1,3.2,70.6,186.3,63,13,10.3,49.6208
4,2010-05,강원,16.1,21.9,10.4,149.5,208.6,67,22,10.0,60.4391
...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-03,충북,7.0,13.9,0.0,23.6,244.1,62,14,4.5,47.4006
2155,2020-04,충북,10.2,17.3,2.9,39.7,286.3,56,11,3.5,52.2089
2156,2020-05,충북,17.6,23.6,12.0,130.1,229.5,72,20,9.5,62.8053
2157,2020-06,충북,23.0,28.5,18.2,182.8,204.0,75,25,12.5,71.2825


In [ ]:
dp_all['REG_YYMM']=dp_all['REG_YYMM'].str.replace('-','').astype(int)
dp_all.to_csv('/content/drive/My Drive/data/weather2/K_WEATHER.csv'.format(),index=False, encoding='utf-8-sig')  

## 미세먼지
택윤 씨 코드  
출처 : https://www.airkorea.or.kr/web/sidoQualityCompare?itemCode=10008&pMENU_NO=102
(2020 3월 4월)

https://kosis.kr/search/search.do  
(2020년 3월 이전)

In [ ]:
# coding: utf-8

# In[2]:

import pandas as pd
import numpy as np
import os

os.chdir('/content/drive/My Drive/data/dust')

finedust =  pd.read_csv('미세먼지_도시별.csv', encoding = 'euc-kr')

a = list(map( lambda x : x.replace('. ', '')  ,  list(finedust.columns)[1:]))#.insert(0, '도시')
a.insert(0, '도시')

finedust.columns = a 

header = finedust.T.iloc[0]

finedust = finedust.T[1:]
finedust.rename(columns=header, inplace=True)

for i in finedust.columns :
    finedust[i] = finedust[i].astype(float)
    
finedust['세종'].fillna(finedust['대전'], inplace = True)
finedust['충남'].fillna(finedust['세종'], inplace = True)

#2020년 3,4월은 에어코리아
# 그이전은 

dust2003 = pd.read_csv('미세먼지_2020-03.csv', encoding = 'euc-kr') 
dust2004 = pd.read_csv('미세먼지_2020-04.csv', encoding = 'euc-kr') 

dust2003.drop(['날짜'], axis =1 , inplace =True)
dust2004.drop(['날짜'], axis =1 , inplace =True)

dust2003= pd.DataFrame(dust2003.T.mean(axis =1))
dust2004= pd.DataFrame(dust2004.T.mean(axis =1))

dust2003.columns = ['202003']
dust2004.columns = ['202004']

dust2003 = dust2003.T
dust2004 = dust2004.T

dust2003 = dust2003[ list(finedust.columns) ]
dust2004 = dust2004[ list(finedust.columns) ]

integ = pd.concat([finedust, dust2003])
integ = pd.concat([integ, dust2004])

integ = integ.reset_index()
integ = pd.melt(integ, id_vars=['index'])
integ.columns=['REG_YYMM','CARD_SIDO_NM','DUST']
integ.to_csv('/content/drive/My Drive/data/dust/dust.csv')

In [ ]:
integ

,REG_YYMM,CARD_SIDO_NM,DUST
0,201001,서울,59.000000
1,201002,서울,50.000000
2,201003,서울,61.000000
3,201004,서울,49.000000
4,201005,서울,56.000000
...,...,...,...
2103,201912,제주,29.000000
2104,202001,제주,31.000000
2105,202002,제주,34.000000
2106,202003,제주,35.032258


## 코로나
동혁  
출처: https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030#

In [ ]:
d_corona=pd.read_excel('/content/drive/My Drive/data/COVID_CONFIRMED(modified).xlsx')

In [ ]:
d_corona.columns=['REG_YYMM','CARD_SIDO_NM','COV_TOTAL','COV_NEW']

In [ ]:
d_corona

,REG_YYMM,CARD_SIDO_NM,COV_TOTAL,COV_NEW
0,202001,경기,2,2
1,202001,서울,7,7
2,202001,인천,1,1
3,202002,강원,6,6
4,202002,경기,66,64
...,...,...,...,...
83,202006,전남,24,5
84,202006,전북,27,6
85,202006,제주,19,4
86,202006,충남,167,21


## 외부데이터 불러오기

In [ ]:
#기후
dp_all=pd.read_csv('/content/drive/My Drive/data/weather2/K_WEATHER.csv')

In [ ]:
#미세먼지
dust=pd.read_csv('/content/drive/My Drive/data/dust/dust.csv',index_col=0)

In [ ]:
#코로나
d_corona=pd.read_excel('/content/drive/My Drive/data/COVID_CONFIRMED(modified).xlsx')
d_corona.columns=['REG_YYMM','CARD_SIDO_NM','COV_TOTAL','COV_NEW']

In [ ]:
dp_all['CARD_SIDO_NM'].unique()

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '서울', '세종', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [ ]:
dust['CARD_SIDO_NM'].unique()

array(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북',
       '충남', '전북', '전남', '경북', '경남', '제주'], dtype=object)

In [ ]:
d_corona['CARD_SIDO_NM'].unique()

array(['경기', '서울', '인천', '강원', '경남', '경북', '광주', '대구', '대전', '부산', '세종',
       '울산', '전남', '전북', '제주', '충남', '충북'], dtype=object)

## 데이터 합치기

In [ ]:
data_jj=data_jj.merge(dp_all[dp_all['REG_YYMM']>201900],
                      on=['REG_YYMM','CARD_SIDO_NM'],how='outer')\
                      .merge(dust[dust['REG_YYMM']>201900],
                             on=['REG_YYMM','CARD_SIDO_NM'],how='outer')\
                             .merge(d_corona,
                                    on=['REG_YYMM','CARD_SIDO_NM'],how='outer')

In [ ]:
data_jj=data_jj[data_jj['REG_YYMM']<202004]

In [ ]:
data_jj

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE,DUST,COV_TOTAL,COV_NEW
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1.0,1.0,4.0,311200.0,4.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.000000,NaN,NaN
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1.0,2.0,7.0,1374500.0,8.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.000000,NaN,NaN
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2.0,2.0,6.0,818700.0,6.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.000000,NaN,NaN
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1.0,3.0,4.0,1717000.0,5.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.000000,NaN,NaN
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1.0,4.0,3.0,1047300.0,3.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1.0,2.0,3.0,43300.0,4.0,7.0,13.9,0.0,23.6,244.1,62.0,14,4.5,47.40060,42.903226,44.0,35.0
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1.0,3.0,3.0,35000.0,3.0,7.0,13.9,0.0,23.6,244.1,62.0,14,4.5,47.40060,42.903226,44.0,35.0
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1.0,4.0,4.0,188000.0,6.0,7.0,13.9,0.0,23.6,244.1,62.0,14,4.5,47.40060,42.903226,44.0,35.0
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2.0,4.0,4.0,99000.0,6.0,7.0,13.9,0.0,23.6,244.1,62.0,14,4.5,47.40060,42.903226,44.0,35.0


In [ ]:
data_jj.columns

Index(['REG_YYMM', 'CARD_SIDO_NM', 'CARD_CCG_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM',
       'HOM_CCG_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'CSTMR_CNT', 'AMT', 'CNT',
       'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM',
       'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE', 'DUST', 'COV_TOTAL',
       'COV_NEW'],
      dtype='object')

## 예측 데이터 만들기 by lstm

In [ ]:
# api 예측을 위한 MinMaxScaling
minmax_ta=MinMaxScaler()
minmax_tt=MinMaxScaler()
minmax_tb=MinMaxScaler()
minmax_ra=MinMaxScaler()
minmax_sh=MinMaxScaler()
minmax_ha=MinMaxScaler()
minmax_hm=MinMaxScaler()
minmax_rd=MinMaxScaler()
minmax_thi=MinMaxScaler()
#아래는 미세먼지
minmax_ds=MinMaxScaler()

In [ ]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back)])
        dataY.append(signal_data[i + look_back])
    return np.array(dataX), np.array(dataY)

class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
look_back = 12

In [ ]:
#스케일링
dp_all['TEMP_AVG']=minmax_ta.fit_transform(dp_all['TEMP_AVG'].values.reshape(-1,1))
dp_all['TEMP_TOP']=minmax_tt.fit_transform(dp_all['TEMP_TOP'].values.reshape(-1,1))
dp_all['TEMP_BOT']=minmax_tb.fit_transform(dp_all['TEMP_BOT'].values.reshape(-1,1))
dp_all['RAIN_AVG']=minmax_ra.fit_transform(dp_all['RAIN_AVG'].values.reshape(-1,1))
dp_all['SUN_HOUR_SUM']=minmax_sh.fit_transform(dp_all['SUN_HOUR_SUM'].values.reshape(-1,1))
dp_all['HUMID_AVG']=minmax_ha.fit_transform(dp_all['HUMID_AVG'].values.reshape(-1,1))
dp_all['HUMID_MIN']=minmax_hm.fit_transform(dp_all['HUMID_MIN'].values.reshape(-1,1))
dp_all['RAIN_DAY']=minmax_rd.fit_transform(dp_all['RAIN_DAY'].values.reshape(-1,1))
dp_all['THI_SCORE']=minmax_thi.fit_transform(dp_all['THI_SCORE'].values.reshape(-1,1))

In [ ]:
#4월 아래 데이터만 사용.
dp_all=dp_all[dp_all['REG_YYMM']<202005]

In [ ]:
a='강릉'
b= [[1,2],[3,4]]
np.concatenate()

In [ ]:
sido_list=list(dp_all['CARD_SIDO_NM'].unique())

In [ ]:
#기후 예측 코드

pred_w=pd.DataFrame(columns=['REG_YYMM', 'CARD_SIDO_NM', 'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT',
       'RAIN_AVG', 'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY',
       'THI_SCORE'])
for sido in tqdm(sido_list):
  np.random.seed(7)
  signal_data=dp_all[dp_all['CARD_SIDO_NM']==sido].iloc[:,2:].values
  x_train, y_train = create_dataset(signal_data, look_back)
  model = Sequential()
  for i in range(2):
    model.add(LSTM(36, batch_input_shape=(1, look_back, 9), stateful=True, return_sequences=True))
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.3))
  model.add(LSTM(24, batch_input_shape=(1, look_back, 9), stateful=True)) 
  model.add(Dropout(0.3))
  model.add(Dense(9))
  model.compile(loss='mean_squared_error', optimizer='adam')
  # 4. 모델 학습시키기
  custom_hist = CustomHistory()
  custom_hist.init()
  for i in range(300):
    model.fit(x_train, y_train, epochs=2, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_split=0.2,verbose=0)
    model.reset_states()
  d4=np.concatenate((x_train[-1][1:],y_train[-1].reshape(1,9)))
  w5=model.predict(np.array([d4]))
  d5=np.concatenate((d4[1:],w5))
  w6=model.predict(np.array([d5]))
  d6=np.concatenate((d5[1:],w6))
  w7=model.predict(np.array([d6]))
  p_data=np.concatenate((d6[-3:],w7))
  p_pd=pd.DataFrame(p_data,columns=['TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT',
       'RAIN_AVG', 'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY',
       'THI_SCORE'])
  p_pd['REG_YYMM']=[202004,202005,202006,202007]
  p_pd['CARD_SIDO_NM']=sido
  p_pd=p_pd[['REG_YYMM', 'CARD_SIDO_NM', 'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT',
       'RAIN_AVG', 'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY',
       'THI_SCORE']]
  p_pd['TEMP_AVG']=minmax_ta.inverse_transform(p_pd['TEMP_AVG'].values.reshape(-1,1))
  p_pd['TEMP_TOP']=minmax_tt.inverse_transform(p_pd['TEMP_TOP'].values.reshape(-1,1))
  p_pd['TEMP_BOT']=minmax_tb.inverse_transform(p_pd['TEMP_BOT'].values.reshape(-1,1))
  p_pd['RAIN_AVG']=minmax_ra.inverse_transform(p_pd['RAIN_AVG'].values.reshape(-1,1))
  p_pd['SUN_HOUR_SUM']=minmax_sh.inverse_transform(p_pd['SUN_HOUR_SUM'].values.reshape(-1,1))
  p_pd['HUMID_AVG']=minmax_ha.inverse_transform(p_pd['HUMID_AVG'].values.reshape(-1,1))
  p_pd['HUMID_MIN']=minmax_hm.inverse_transform(p_pd['HUMID_MIN'].values.reshape(-1,1))
  p_pd['RAIN_DAY']=minmax_rd.inverse_transform(p_pd['RAIN_DAY'].values.reshape(-1,1))
  p_pd['THI_SCORE']=minmax_thi.inverse_transform(p_pd['THI_SCORE'].values.reshape(-1,1))
  pred_w=pd.concat([pred_w,p_pd])


In [ ]:
pred_w

,REG_YYMM,CARD_SIDO_NM,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE
0,202004,강원,6.500000,12.900000,0.200000,48.800000,273.600000,70.000000,19.000000,6.700000,46.059500
1,202005,강원,17.947977,23.955204,12.780556,167.179128,252.998848,70.215786,20.564909,8.070588,64.682839
2,202006,강원,16.987985,22.835728,12.003928,187.902954,244.874028,71.794094,20.067074,8.870017,63.286726
3,202007,강원,19.179675,25.015859,14.043218,170.608085,264.513904,67.803440,23.785989,8.356835,66.673575
0,202004,경기,6.900000,13.900000,0.200000,20.400000,262.300000,71.000000,25.000000,4.800000,46.586010
...,...,...,...,...,...,...,...,...,...,...,...
3,202007,충남,-0.836834,5.397111,-6.916760,-15.455468,212.823560,68.901134,13.629504,6.736799,34.063590
0,202004,충북,7.000000,13.900000,0.000000,23.600000,244.100000,62.000000,14.000000,4.500000,47.400600
1,202005,충북,22.236401,26.975542,18.777311,142.323535,180.712680,83.189957,38.426221,11.659404,70.233116
2,202006,충북,11.013923,17.852551,4.626895,96.703724,261.611318,64.535372,15.963626,7.556002,53.071724


In [ ]:
pred_w

,REG_YYMM,CARD_SIDO_NM,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE
0,202004,강원,6.500000,12.900000,0.200000,48.800000,273.600000,70.000000,19.000000,6.700000,46.059500
1,202005,강원,6.858682,12.575013,1.338589,95.172847,229.551057,66.462273,17.495993,10.006616,46.670547
2,202006,강원,11.518881,17.780475,5.425024,65.148750,273.294025,62.988003,15.273496,8.642142,53.964991
3,202007,강원,17.566836,23.422527,12.406183,130.121564,246.133000,77.358017,24.644500,10.517840,62.831110
0,202004,경기,6.900000,13.900000,0.200000,20.400000,262.300000,71.000000,25.000000,4.800000,46.586010
...,...,...,...,...,...,...,...,...,...,...,...
3,202007,충남,19.402050,24.522030,15.130935,292.680630,172.725575,85.619823,37.341736,12.040441,66.945675
0,202004,충북,7.000000,13.900000,0.000000,23.600000,244.100000,62.000000,14.000000,4.500000,47.400600
1,202005,충북,19.312966,24.225620,15.411777,288.022026,166.683961,75.899208,27.747165,17.764365,66.737010
2,202006,충북,19.163045,25.872088,12.992188,100.887086,262.563912,63.469416,17.338585,8.464872,65.580002


In [ ]:
pred_w.to_csv('/content/drive/My Drive/data/w_pred_lstm.csv')

In [ ]:
dust['DUST']=minmax_ds.fit_transform(dust['DUST'].values.reshape(-1,1))

In [ ]:
# 미세먼지 예측 코드

pred_d=pd.DataFrame(columns=['REG_YYMM', 'CARD_SIDO_NM', 'DUST'])
for sido in tqdm(sido_list):
  np.random.seed(8)
  signal_data=dust[dust['CARD_SIDO_NM']==sido].iloc[:,2:].values
  x_train, y_train = create_dataset(signal_data, look_back)
  model = Sequential()
  for i in range(2):
    model.add(LSTM(36, batch_input_shape=(1, look_back, 1), stateful=True, return_sequences=True))
    model.add(keras.layers.BatchNormalization())
    model.add(Dropout(0.3))
  model.add(LSTM(24, batch_input_shape=(1, look_back, 1), stateful=True)) 
  model.add(Dropout(0.3))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error', optimizer='adam')
  # 4. 모델 학습시키기
  custom_hist = CustomHistory()
  custom_hist.init()
  for i in range(300):
    model.fit(x_train, y_train, epochs=2, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_split=0.2,verbose=0)
    model.reset_states()
  d4=np.concatenate((x_train[-1][1:],y_train[-1].reshape(1,1)))
  w5=model.predict(np.array([d4]))
  d5=np.concatenate((d4[1:],w5))
  w6=model.predict(np.array([d5]))
  d6=np.concatenate((d5[1:],w6))
  w7=model.predict(np.array([d6]))
  p_data=np.concatenate((d6[-3:],w7))
  p_pd=pd.DataFrame(p_data,columns=['DUST'])
  p_pd['REG_YYMM']=[202004,202005,202006,202007]
  p_pd['CARD_SIDO_NM']=sido
  p_pd=p_pd[['REG_YYMM', 'CARD_SIDO_NM','DUST']]
  p_pd['DUST']=minmax_ds.inverse_transform(p_pd['DUST'].values.reshape(-1,1))
  pred_d=pd.concat([pred_d,p_pd])


In [ ]:
pred_d

,REG_YYMM,CARD_SIDO_NM,DUST
0,202004,강원,18.800000
1,202005,강원,59.280808
2,202006,강원,39.991991
3,202007,강원,49.062317
0,202004,경기,20.200000
...,...,...,...
3,202007,충남,48.679775
0,202004,충북,21.133333
1,202005,충북,56.348371
2,202006,충북,46.679003


In [ ]:
pred_d.to_csv('/content/drive/My Drive/data/d_pred_lstm.csv')

# 데이터 전처리 클래스

In [ ]:
class x_y_maker():
  def __init__(self,data):
    self.data=data
    #데이터 카피
    self.ori_data=data.copy()
    #AGE->AGE_GROUP
    self.ori_data['AGE_GROUP']=self.ori_data['AGE'].str.extract('([0-9])').astype('int')
    #SEX_AGE 병합
    self.ori_data['SEX_AGE']=self.ori_data['SEX_CTGO_CD']+2*(self.ori_data['AGE_GROUP']-1)
    #시도간 이동 라벨링
    self.ori_data['SI_CC_LB']=0
    self.ori_data['SI_CC_LB']=self.ori_data['SI_CC_LB'].where(self.ori_data['CARD_CCG_NM']==self.ori_data['HOM_CCG_NM'],1)
    self.ori_data['SI_CC_LB']=self.ori_data['SI_CC_LB'].where(self.ori_data['CARD_SIDO_NM']==self.ori_data['HOM_SIDO_NM'],2)
    #시도, 업종 라벨 인코딩
    self.encoder_s=LabelEncoder()
    self.encoder_c=LabelEncoder()
    #CSTMR_CNT 와 CNT 차이 라벨링
    self.ori_data['REFUND_LB']=np.where(self.ori_data['CSTMR_CNT']>self.ori_data['CNT'],1,0)
    # 치우쳐진 데이터는 log1p 변환후 scaling 해주는 게 보통이라고 한다.
    self.stscaler=StandardScaler()

#특성 더 넣고 싶은 게 있으면 여기서 칼럼 만들어주면 된다.
#잘 모르겠으면 그냥 물어봐줘...
  
  def col_setting(self,full_data):
    #STD_CLSS_LB 만들기 (업종 데이터 라벨화)
    sample_label_s=self.encoder_s.transform(full_data['STD_CLSS_NM'])
    full_data['STD_CLSS_LB']=sample_label_s
    #CARD_SIDO_LB 만들기 (시도 데이터 라벨화)
    sample_label_c=self.encoder_c.transform(full_data['CARD_SIDO_NM'])
    full_data['CARD_SIDO_LB']=sample_label_c
    # MM, YY 나눠주기
    full_data['MM']=full_data['REG_YYMM']%100
    full_data['YY']=full_data['REG_YYMM']//100
    #필요한 칼럼만 축소
    
    actual_data=full_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','FLC','SEX_AGE','SI_CC_LB','REFUND_LB',
                           'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE',
                           'DUST', 'COV_TOTAL', 'COV_NEW','AMT']]    
    return actual_data 

    # 원데이터 변형
  def data_maker(self):
    # CARD_CCG_NM, HOM_SIDO_NM, HOM_CCG_NM, SEX_CTGO, AGE, AGE_GROUP, CSTMR_CNT, CNT 삭제
    # CSTMR_CNT, CNT를 SUB에 반영할 수가 없다. ㅠ
    # 원데이터에서는 따로 NaN을 집어넣어주지 않음. NaN 때문에 학습이 안되서.
    # 만약 결측치 처리한 게 있으면 원데이터 대신 넣어주면 된다.
    mak_data=self.ori_data[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC','SEX_AGE', 'SI_CC_LB','REFUND_LB',
                            'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE',
                            'DUST', 'COV_TOTAL', 'COV_NEW',
                       'AMT']].groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'SEX_AGE', 'SI_CC_LB','REFUND_LB',
                                        'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY','THI_SCORE',
                                        'DUST', 'COV_TOTAL', 'COV_NEW']).agg(['sum']).reset_index()
    mak_data.columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'FLC', 'SEX_AGE', 'SI_CC_LB','REFUND_LB',
                      'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE',
                      'DUST', 'COV_TOTAL', 'COV_NEW','AMT']
    #라벨 인코딩
    self.encoder_s.fit(mak_data['STD_CLSS_NM'])
    self.encoder_c.fit(mak_data['CARD_SIDO_NM'])
    t_data=self.col_setting(mak_data)
    # 칼럼 골라쓰고 싶으면 아래에서 원하는 칼럼 골라서 넣어주면 된다.
    # t_data=t_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB', 'FLC','SEX_AGE', 'SI_CC_LB','REFUND_LB',
    #                'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 
    #                'DUST', 'COV_TOTAL', 'COV_NEW',
    #                'AMT']].groupby(['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB', 'FLC', 'SEX_AGE', 'SI_CC_LB','REFUND_LB',
    #                                 'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY',
    #                                 'DUST', 'COV_TOTAL', 'COV_NEW']).agg(['sum']).reset_index()
    # 칼럼 골라쓰고 싶으면 위의 칼럼과 여기 칼럼을 매치 시켜줘.
    X_t=t_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','FLC','SEX_AGE','SI_CC_LB','REFUND_LB',
                'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY','THI_SCORE',
                'DUST', 'COV_TOTAL', 'COV_NEW']]
    y_ft=t_data[['AMT']]
    #log 변환 후 표준화
    y_ts=self.stscaler.fit_transform(np.log1p(y_ft))
    #shape 맞춰주기
    y_t=y_ts.flatten()
    return X_t, y_t

    # sub데이터 변형
  def sub_maker(self,sub):
    month_list = list(set(sub['REG_YYMM']))
    sido_list = list(set(sub['CARD_SIDO_NM']))
    std_list=list(set(sub['STD_CLSS_NM']))
    flc_list=list(set(self.ori_data['FLC']))
    sage_list=list(set(self.ori_data['SEX_AGE']))
    sicc_list=list(set(self.ori_data['SI_CC_LB']))
    refund_list=list(set(self.ori_data['REFUND_LB']))
    sub_nan =[]
    for month in month_list:
      for sido in sido_list:
        for std in std_list:
          for flc in flc_list:
            for sage in sage_list:
              for sicc in sicc_list:
                for ref in refund_list:
                  sub_nan.append([int(month),sido,std,flc,sage,sicc,ref,
                                  np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
                                  np.nan,np.nan,np.nan,np.nan])
    sub_nans=pd.DataFrame(sub_nan,columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM','FLC','SEX_AGE','SI_CC_LB','REFUND_LB',
                                           'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG', 'SUN_HOUR_SUM','HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY','THI_SCORE',
                                           'DUST', 'COV_TOTAL', 'COV_NEW','AMT'])
    sub_data=self.col_setting(sub_nans)
    X_s=sub_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','FLC','SEX_AGE','SI_CC_LB','REFUND_LB']]
    y_fs=sub_data[['AMT']]
    #shape 맞춰주기
    y_s=y_fs.values.flatten()
    return X_s, y_s

    # sub데이터 디코드, sub_t에는 서브미션에서 AMT제외한 거, sub_p에는 predict value 넣어주면 됨.
  def sub_decoder(self,sub_t,sub_p):
    sub_tt=sub_t.copy()
    sub_tt['STD_CLSS_NM']=self.encoder_s.inverse_transform(sub_tt['STD_CLSS_LB'])
    sub_tt['CARD_SIDO_NM']=self.encoder_c.inverse_transform(sub_tt['CARD_SIDO_LB'])
    sub_tt['REG_YYMM']=sub_tt['YY']*100+sub_tt['MM']
    #invers scaling
    st_p=self.stscaler.inverse_transform(sub_p)
    #log 변환 되돌려주기
    fin_p= np.expm1(st_p)
    sub_tt['AMT']=np.round(fin_p,0)
    fin_sub=sub_tt[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']].groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).agg(['sum']).reset_index()
    fin_sub.columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']
    fin_sub.index.name = 'id'
    return fin_sub

In [ ]:
# 코랩용 풀데이터 불러오기
data_jj=pd.read_csv('/content/drive/My Drive/data/full_data.csv')

In [ ]:
# 객체 생성. 여기서 data를 넣어줌.
xy_maker = x_y_maker(data_jj)

In [ ]:
# 객체 내 메서드 실행 => 원 데이터 변형
x,y=xy_maker.data_maker()

In [ ]:
# 객체 내 메서드 실행 => sub 데이터 변형
X_sub, y_sub=xy_maker.sub_maker(sub)

In [ ]:
pred_d

,REG_YYMM,CARD_SIDO_NM,DUST
0,202004,강원,18.800000
1,202005,강원,59.280808
2,202006,강원,39.991991
3,202007,강원,49.062317
0,202004,경기,20.200000
...,...,...,...
3,202007,충남,48.679775
0,202004,충북,21.133333
1,202005,충북,56.348371
2,202006,충북,46.679003


In [ ]:
pred_w

,REG_YYMM,CARD_SIDO_NM,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE
0,202004,강원,6.500000,12.900000,0.200000,48.800000,273.600000,70.000000,19.000000,6.700000,46.059500
1,202005,강원,17.947977,23.955204,12.780556,167.179128,252.998848,70.215786,20.564909,8.070588,64.682839
2,202006,강원,16.987985,22.835728,12.003928,187.902954,244.874028,71.794094,20.067074,8.870017,63.286726
3,202007,강원,19.179675,25.015859,14.043218,170.608085,264.513904,67.803440,23.785989,8.356835,66.673575
0,202004,경기,6.900000,13.900000,0.200000,20.400000,262.300000,71.000000,25.000000,4.800000,46.586010
...,...,...,...,...,...,...,...,...,...,...,...
3,202007,충남,-0.836834,5.397111,-6.916760,-15.455468,212.823560,68.901134,13.629504,6.736799,34.063590
0,202004,충북,7.000000,13.900000,0.000000,23.600000,244.100000,62.000000,14.000000,4.500000,47.400600
1,202005,충북,22.236401,26.975542,18.777311,142.323535,180.712680,83.189957,38.426221,11.659404,70.233116
2,202006,충북,11.013923,17.852551,4.626895,96.703724,261.611318,64.535372,15.963626,7.556002,53.071724


In [ ]:
pred_w['YY']=pred_w['REG_YYMM']//100, pred_w['MM']=pred_w['REG_YYMM']%100
pred_d['YY']=pred_d['REG_YYMM']//100, pred_d['MM']=pred_d['REG_YYMM']%100

In [ ]:
pred_a=pd.concat([pred_w,pred_d],axis=1)

In [ ]:
X_sub=X_sub.merge(pred_a,left_on=['YY','MM'])

In [ ]:
x.head()

,YY,MM,CARD_SIDO_LB,STD_CLSS_LB,FLC,SEX_AGE,SI_CC_LB,REFUND_LB,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE,DUST,COV_TOTAL,COV_NEW
0,2020,1,1,0,1.0,1.0,0,0,0.6,6.1,-3.9,82.3,193.5,78.0,30,5.3,36.09532,47.0,2.0,2.0
1,2020,1,1,0,1.0,2.0,0,0,0.6,6.1,-3.9,82.3,193.5,78.0,30,5.3,36.09532,47.0,2.0,2.0
2,2020,1,1,0,1.0,3.0,0,0,0.6,6.1,-3.9,82.3,193.5,78.0,30,5.3,36.09532,47.0,2.0,2.0
3,2020,1,1,0,1.0,3.0,0,1,0.6,6.1,-3.9,82.3,193.5,78.0,30,5.3,36.09532,47.0,2.0,2.0
4,2020,1,1,0,1.0,3.0,1,0,0.6,6.1,-3.9,82.3,193.5,78.0,30,5.3,36.09532,47.0,2.0,2.0


In [ ]:
X_sub.head()

,YY,MM,CARD_SIDO_LB,STD_CLSS_LB,FLC,SEX_AGE,SI_CC_LB,REFUND_LB,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE,DUST,COV_TOTAL,COV_NEW
0,2020,4,11,10,1.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,4,11,10,1.0,1.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,4,11,10,1.0,1.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,4,11,10,1.0,1.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,4,11,10,1.0,1.0,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
y

array([-1.73102264, -2.20617001,  0.40549899, ..., -1.26617913,
       -1.83129262, -0.6825077 ])

In [ ]:
y_sub

array([nan, nan, nan, ..., nan, nan, nan])

In [ ]:
x.columns

Index(['YY', 'MM', 'CARD_SIDO_LB', 'STD_CLSS_LB', 'FLC', 'SEX_AGE', 'SI_CC_LB',
       'REFUND_LB', 'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG',
       'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE',
       'DUST', 'COV_TOTAL', 'COV_NEW'],
      dtype='object')

In [ ]:
X_sub.columns

Index(['YY', 'MM', 'CARD_SIDO_LB', 'STD_CLSS_LB', 'FLC', 'SEX_AGE', 'SI_CC_LB',
       'REFUND_LB', 'TEMP_AVG', 'TEMP_TOP', 'TEMP_BOT', 'RAIN_AVG',
       'SUN_HOUR_SUM', 'HUMID_AVG', 'HUMID_MIN', 'RAIN_DAY', 'THI_SCORE',
       'DUST', 'COV_TOTAL', 'COV_NEW'],
      dtype='object')

In [ ]:
data_jj.head()

,Unnamed: 0,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,TEMP_AVG,TEMP_TOP,TEMP_BOT,RAIN_AVG,SUN_HOUR_SUM,HUMID_AVG,HUMID_MIN,RAIN_DAY,THI_SCORE,DUST,COV_TOTAL,COV_NEW
0,0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1.0,1.0,4.0,311200.0,4.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.0,NaN,NaN
1,1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1.0,2.0,7.0,1374500.0,8.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.0,NaN,NaN
2,2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2.0,2.0,6.0,818700.0,6.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.0,NaN,NaN
3,3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1.0,3.0,4.0,1717000.0,5.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.0,NaN,NaN
4,4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1.0,4.0,3.0,1047300.0,3.0,-1.8,4.4,-7.6,17.6,240.6,59.0,16,1.7,35.35362,59.0,NaN,NaN


In [ ]:
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,0
1,1,202004,강원,골프장 운영업,0
2,2,202004,강원,과실 및 채소 소매업,0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0


In [ ]:
#얘네는 쓰려면 또 알아서 쓰면 됨.
dp_all.head()

In [ ]:
dust.head()

In [ ]:
d_corona.head()

In [ ]:
isolmodel = IsolationForest(n_estimators=200,max_samples='auto',n_jobs=-1,max_features=2,contamination='auto',random_state=222)

In [ ]:
  #여기서 이렇게 모델 나눠줘도 되고/ train 데이터를 불러와도 된다.
#만약 train 데이터를 불러온다면 train/test 가아니라 (train/validation)/test 라고 볼 수 있다.
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=777,shuffle = True)

In [ ]:
import xgboost as xgb

In [ ]:
#gridsearch 예시
# 아래 xgb2 = xgb.XGBRegressor()에 원하는 모델 넣어주고
# 파라미터에 그 모델 파라미터 시험하고 싶은 거 저렇게 리스트로 넣어준 뒤에
# gridsearchCV에 (모델명, 파라미터, cv=?,n_jobs=?) 맞춰서 넣어주면 된다.
# 파라미터는 원하는 모델에 gridsearchCV 치면 잘 나올듯
# ex) XGB gridsearchCV
# cv는 Kfold(n) 값이란다. 2나 5 쓰는 듯.
# 그리고 grid 모형에 fit 해주고 아래에서 best_score, best_params를 뽑자.

# 이거 예시니까 돌리지마요. 돌리면 하루종일 돌아감.

xgb2 = xgb.XGBRegressor()
parameters = {'booster':['gbtree','gblinear','dart'],
              'learning_rate': [0.01,0.03, 0.05,0.1], #so called `eta` value
              'max_depth': [4, 5, 6],
              'min_child_weight': [2,3,4,5],
              'subsample': [0.7],
              'lambda':[0.1,0.5,1,10],
              'alpha':[0.1,0.5,1,10],
              'colsample_bytree': [0.7],
              'n_estimators': [1000]}

xgb_grid = GridSearchCV(xgb2,
                        parameters,
                        cv = 2,
                        n_jobs = -1,
                        verbose=2)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [ ]:
# subdecoder에 X_sub(sub maker로 만든 X와 predict한 AMT 배열을 넣어주면 최종 파일로 만들어줌.)
# submission=xy_maker.sub_decoder(X_sub,xgb_grid.predict(y_sub))
submission=xy_maker.sub_decoder(X_sub,y_sub)

In [ ]:
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,0.0
1,202004,강원,골프장 운영업,0.0
2,202004,강원,과실 및 채소 소매업,0.0
3,202004,강원,관광 민예품 및 선물용품 소매업,0.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0


In [ ]:
sub.head()

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,0
1,1,202004,강원,골프장 운영업,0
2,2,202004,강원,과실 및 채소 소매업,0
3,3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0


In [ ]:
# 이런 식으로 저장.
# submission.to_csv('',index=False)